# Technical Factor - Money Flow Index (MFI)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (high + low + close) / 3 AS TP,
        IF(TP > m_lag(TP, 1), TP * volume, 0) AS PMF,
        IF(TP < m_lag(TP, 1), TP * volume, 0) AS NMF,
        m_sum(PMF, 14) AS PMF_Total,
        m_sum(NMF, 14) AS NMF_Total,
        PMF_Total / (NMF_Total + 0.0001) AS MFR,
        100 - (100 / (1 + MFR)) AS MFI,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(MFI > 50 AND m_lag(MFI, 1) <= 50, 1, 0) AS TRBY1,
        IF(MFI < 50 AND m_lag(MFI, 1) >= 50, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(MFI > m_lag(MFI, 1) AND m_lag(MFI, 1) >= m_lag(MFI, 2), 1, 0) AS MTBY1,
        IF(MFI < m_lag(MFI, 1) AND m_lag(MFI, 1) <= m_lag(MFI, 2), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(MFI < 20 AND (MFI - m_lag(MFI, 1)) > 0, 1, 0) AS RVBY1,
        IF(MFI > 80 AND (MFI - m_lag(MFI, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(MFI, 1) <= 80 AND MFI > 80, 1, 0) AS BKBY1,
        IF(m_lag(MFI, 1) >= 20 AND MFI < 20, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        TP,
        PMF,
        NMF,
        PMF_Total,
        NMF_Total,
        MFR,
        MFI,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,TP,PMF,NMF,PMF_Total,NMF_Total,MFR,MFI,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-21,600519.SH,11988.472355,0.000000e+00,2.020782e+10,1.316942e+11,2.847731e+11,0.462453,31.621734,0,0,0,0,0,0,0,0
1,2025-01-22,600519.SH,11803.288655,0.000000e+00,3.339604e+10,1.316942e+11,3.181691e+11,0.413912,29.274264,0,0,0,0,0,0,0,0
2,2025-01-23,600519.SH,11808.855027,3.203157e+10,0.000000e+00,1.637257e+11,2.788591e+11,0.587127,36.993072,0,0,0,0,0,0,0,0
3,2025-01-24,600519.SH,11714.090934,0.000000e+00,3.673579e+10,1.637257e+11,2.634974e+11,0.621356,38.323238,0,0,1,0,0,0,0,0
4,2025-01-27,600519.SH,11692.042670,0.000000e+00,3.432370e+10,1.637257e+11,2.693364e+11,0.607886,37.806519,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,2025-12-25,600519.SH,11924.421463,2.788670e+10,0.000000e+00,2.203321e+11,2.467760e+11,0.892843,47.169404,0,0,0,0,0,0,0,0
226,2025-12-26,600519.SH,11947.622535,2.127034e+10,0.000000e+00,2.416025e+11,2.070253e+11,1.167019,53.853657,1,0,0,0,0,0,0,0
227,2025-12-29,600519.SH,11874.021078,0.000000e+00,3.123829e+10,2.416025e+11,1.884288e+11,1.282195,56.182530,0,0,1,0,0,0,0,0
228,2025-12-30,600519.SH,11762.774194,0.000000e+00,3.974870e+10,2.416025e+11,1.897536e+11,1.273243,56.009988,0,0,0,0,0,0,0,0
